In [1]:
#!conda install -n mldds -c anaconda joblib
import os
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='once')

import multiprocessing
num_cores = multiprocessing.cpu_count()

print("Cores: ", num_cores)

import time
import keras
import tensorflow as tf
config = tf.ConfigProto( device_count = {'GPU': 0 , 'CPU': num_cores} )
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

Cores:  12


/Users/walter/Software/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
Using TensorFlow backend.
/Users/walter/Software/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/ops.py:923: DeprecationWarning: builtin type EagerTensor has no __module__ attribute
  EagerTensor = c_api.TFE_Py_InitEagerTensor(_EagerTensorBase)
/Users/walter/Software/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)
/Users/walter/Software/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/Users/walter/Software/anaconda3/lib/python3.6/site-packages/tensorflow/pyth

In [26]:
import pandas as pd
import numpy as np
from sklearn import utils, preprocessing
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from matplotlib.pyplot import figure
from functools import reduce
from keras.utils import to_categorical
import matplotlib.pyplot as plt


class MyTotoResearch:
    
    @classmethod
    def __init__(self, algo_no=0, inputPPFile='../input/PPv3.csv', inputTotoResult='../input/SGH.csv'):
        self.algo_number = algo_no
        print('Loaded MyTotoResearch algo_no: ', self.algo_number)

    @classmethod
    def load_totodata(self, inputPPFile='../input/PPv3.csv', inputTotoResult='../input/SGH.csv'):
        pp = pd.read_csv(inputPPFile)
        lr = pd.read_csv(inputTotoResult)
        print(len(lr))
        cols = ['D', 'N1','N2','N3','N4','N5','N6','N7']
        lr = lr[cols]

        #https://pandas.pydata.org/pandas-docs/stable/merging.html
        df = pd.concat([pp, lr], axis=1, sort=False)
        df = df.dropna()
        df.reset_index().drop(['D'], axis=1)

#         print(df.shape)
#         counts = [df['N'+str(i)].value_counts() for i in range(1,8)]
#         for i in range(len(counts)):
#             df = df[~df['N'+str(i+1)].isin(counts[i][counts[i] <= 3].index)]
#         print('After removing numbers that have shown 3 or less times')
#         print(df.shape)

        cols = ['N1','N2','N3','N4','N5','N6','N7']
        lr = df[cols]

        self.df = df
        self.lresult = np.sort(lr.values[:, ::-1])
        return self.lresult, self.df 
    
    @classmethod
    def modified_dataset ( self, dataset ):
        self.dataset = dataset
        return self.dataset
    
    @classmethod
    def get_result_n(self, col_n):
        aa = np.delete(self.lresult, np.s_[col_n:], axis=1)  
        aa = np.delete(aa, np.s_[0:col_n-1], axis=1)  
        return pd.DataFrame(aa, columns=list('N'))

    @classmethod
    def get_result_n_encoded(self, col_n):
        aa = np.delete(self.lresult, np.s_[col_n:], axis=1)  
        aa = np.delete(aa, np.s_[0:col_n-1], axis=1)  
        # 1. INSTANTIATE
        enc = preprocessing.OneHotEncoder()

        # 2. FIT
        enc.fit(aa)

        # 3. Transform
        onehotlabels = enc.transform(aa).toarray()
        onehotlabels.shape
        #print(onehotlabels)

        #Convert 2d array to Dataframe
        y = pd.DataFrame(aa, columns=list('N'))
        y.head()
        y = aa.astype(int).ravel()
    #    print ( y )
        return y
        
    @classmethod
    def get_test_data(self, file_name = '../input/PPv3-Predict.csv' ):
        self.data2Predict = pd.read_csv(file_name)
        self.data2Predict.reset_index()
        return self.data2Predict
    

    @classmethod
    def plot_history(self, history):
        loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
        val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
        acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
        val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]

        if len(loss_list) == 0:
            print('Loss is missing in history')
            return 

        ## As loss always exists
        epochs = range(1,len(history.history[loss_list[0]]) + 1)

        ## Loss
        figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
        plt.figure(1)
        for l in loss_list:
            plt.plot(epochs, history.history[l], 'b', label='Training loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
        for l in val_loss_list:
            plt.plot(epochs, history.history[l], 'g', label='Validation loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))

        plt.title('Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()

        ## Accuracy
        plt.figure(1)
        for l in acc_list:
            plt.plot(epochs, history.history[l], 'r', label='Training accuracy (' + str(format(history.history[l][-1],'.5f'))+')')
        for l in val_acc_list:    
            plt.plot(epochs, history.history[l], 'g', label='Validation accuracy (' + str(format(history.history[l][-1],'.5f'))+')')

        plt.title('Accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.show()

    @classmethod
    def save_model(self, model, predict_number):
        # serialize model to JSON
        model_json = model.to_json()
        with open(str(self.algo_number) + '_' + str(predict_number) + "_model.json", "w") as json_file:
            json_file.write(simplejson.dumps(simplejson.loads(model_json), indent=4))

        # serialize weights to HDF5
        model.save_weights(str(self.algo_number) + '_' + str(predict_number) + "_model.h5")
        print("Saved model to disk ", self.algo_number, " Predict #N ", predict_number)

    @classmethod
    def print_result(self, predicted_values ):
        test_df = pd.read_csv('../input/TestResult.csv', sep='\s+', header=None, names=['D','N1','N2','N3','N4','N5','N6','N7'])
        test_df['D'].replace(regex=True,inplace=True,to_replace=r'-',value=r'')
        test_df['D'] = pd.to_numeric(test_df['D'])

        cols = ['D', 'N1','N2','N3','N4','N5','N6','N7']
        test_df = self.data2Predict.merge(test_df, left_on='T', right_on='D', how='inner')
        test_df = test_df[cols]

        tdfResult = predicted_values.drop(predicted_values.columns[0], axis=1) ;

        actual_result = test_df[cols[1:]].values
        predicted_result = tdfResult.values

        matched = getIntersection(actual_result, predicted_result)

        c = 0
        for i in range(len(matched)):
            print(int(self.data2Predict.loc[i]['T']), ' ', actual_result[i], ' ', predicted_result[i], ' ', matched[c])
            c += 1
        for i in range(c, len(predicted_result)):
            print(int(self.data2Predict.loc[i]['T']), ' Predicted: ', predicted_result[i], ' ')

    @classmethod
    def load_model(self, predict_number):
        json_file = open(str(self.algo_number) + "_" + str(predict_number)+'_model.json', 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        loaded_model = model_from_json(loaded_model_json)

        # load weights into new model
        loaded_model.load_weights(str(self.algo_number) + "_" + str(predict_number)+"_model.h5")
        print("Loaded model from disk " + str(self.algo_number) + "_" + str(predict_number) + "_model" )
        return loaded_model

    @classmethod
    def getTargets(self):
        return np.array([self.get_result_n(i)['N'] for i in range(1,8)]).T

    @classmethod
    def getTarget(self, N):
        return ([self.get_result_n(i)['N'] for i in range(1,8)])[N-1]


    @staticmethod
    def getIntersection(p1, p2):
        return [reduce(np.intersect1d, (p.astype(int), a.astype(int))) for (p,a) in zip(p1, p2)]

    @classmethod
    def print_predictions(self, dfPredictions, result='../input/TestResult.csv'):
        #load the test Toto Results files
        test_df = pd.read_csv(result, sep='\s+', header=None, names=['D','N1','N2','N3','N4','N5','N6','N7'])
        test_df['D'].replace(regex=True,inplace=True,to_replace=r'-',value=r'')
        test_df['D'] = pd.to_numeric(test_df['D'])

        #Merge the Planet Position File with the Toto Results file
        test_df = self.data2Predict.merge(test_df, left_on='T', right_on='D', how='inner')

        #Extract only the Results for all dates
        cols = ['D', 'N1','N2','N3','N4','N5','N6','N7']
        test_df = test_df[cols]

        tdfResult = dfPredictions.drop(dfPredictions.columns[0], axis=1) ;

        actual_result = test_df[cols[1:]].values
        predicted_result = tdfResult.values

        matched = MyTotoResearch.getIntersection(actual_result, predicted_result)

        c = 0
        for i in range(len(matched)):
            print(int(self.data2Predict.loc[i]['T']), ' ', actual_result[i], ' ', predicted_result[i], ' ', matched[c])
            c += 1
        for i in range(c, len(predicted_result)):
            print(int(self.data2Predict.loc[i]['T']), ' Predicted: ', predicted_result[i], ' ')

            
def getAdjustedDataF(df,f):
    #Use only Planet Positions Testing
    cols = ['L','M','S', 'R','E','A','V' ,'J','U','K']
    X = df[cols]
    deg = f
    
#     X['S_3'] = X['S'] // (deg*3)
#     X['L_3'] = X['L'] // (deg*3)
#     X['M_3'] = X['M'] // (deg*3)
#     X['R_3'] = X['R'] // (deg*3)
#     X['E_3'] = X['E'] // (deg*3)
#     X['A_3'] = X['A'] // (deg*3)
#     X['V_3'] = X['V'] // (deg*3)
#     X['J_3'] = X['J'] // (deg*3)
#     X['U_3'] = X['U'] // (deg*3)


#     X['S_2'] = X['S'] // (deg*2)
#     X['L_2'] = X['L'] // (deg*2)
#     X['M_2'] = X['M'] // (deg*2)
#     X['R_2'] = X['R'] // (deg*2)
#     X['E_2'] = X['E'] // (deg*2)
#     X['A_2'] = X['A'] // (deg*2)
#     X['V_2'] = X['V'] // (deg*2)
#     X['J_2'] = X['J'] // (deg*2)
#     X['U_2'] = X['U'] // (deg*2)

    X['S_1'] = X['S'] // (deg)
    X['L_1'] = X['L'] // (deg)
    X['M_1'] = X['M'] // (deg)
    X['R_1'] = X['R'] // (deg)
    X['E_1'] = X['E'] // (deg)
    X['A_1'] = X['A'] // (deg)
    X['V_1'] = X['V'] // (deg)
    X['J_1'] = X['J'] // (deg)
    X['U_1'] = X['U'] // (deg)
   
    X = X.drop(cols, axis=1)
    return X




<>:138: DeprecationWarning: invalid escape sequence \s


In [27]:
from keras.models import Input, Model
import keras
from keras.layers import Dense
import time
from keras.models import Sequential
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint, History
import json as simplejson
from keras import regularizers
from sklearn import preprocessing
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDRegressor, SGDClassifier, PassiveAggressiveClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import r2_score

def getAllData(df):
    drop_cols = ['T', 'D', 'N1','N2','N3','N4','N5','N6','N7','L','M','S','R','E','A','V' ,'J','U','K']
    X = df.drop(drop_cols, axis=1)
    return X


#early_stopping = MyEarlyStopping ( threshold=.99999, monitor='acc', verbose=1 )
#early_stopping_loss = MyEarlyStoppingLoss ( threshold=.005, monitor='loss', verbose=1 )

#Deep Neuro Network
mtr = MyTotoResearch(algo_no=1)
lresult, df = mtr.load_totodata()
X = mtr.modified_dataset(getAllData(df)) #
#target = np.array([mtr.get_result_n(i)['N'] for i in range(1,8)]).T
target = mtr.getTargets() ;
#print(target)
scaler = RobustScaler()
scaler.fit(X)
Z = scaler.transform(X)
                 
#Z = X

#sgd = SGDClassifier(max_iter=200, tol=1e-3, random_state=42, penalty="l2")
sgd =  PassiveAggressiveClassifier(max_iter=12000, random_state=42, tol=0.000001, C=0.001, loss='hinge')
#loss="log", alpha=0.001, penalty="l2", max_iter=2000, tol=1e-3, random_state=42)

multi_target_sgd = MultiOutputClassifier(sgd, n_jobs=7)
print(target.shape)
multi_target_sgd.fit(Z, mtr.getTargets()) 
print(multi_target_sgd.score(Z, mtr.getTargets()))

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
#print("Accuracy:",metrics.accuracy_score(mtr.getTargets(), y_pred))

# multi_target_sgd = sgd
# multi_target_sgd.fit(Z, mtr.getTarget(0)) 
# print(multi_target_sgd.score(Z, mtr.getTarget(0)))

start = time.clock()
#history = multi_target_sgd.fit(X, target) #@, epochs=12000, verbose=0) #, callbacks=[early_stopping]) #, callbacks=[early_stopping]) #, checkpoint])    
print(" Time taken: ", (time.clock() - start),  " ")

# n_iters = [100,2000,5000,10000]
# scores = []
# for n_iter in n_iters:
#     model = SGDClassifier(loss="log", alpha=0.001, penalty="l2", max_iter=n_iter, tol=1e-6, random_state=42)
#     model.fit(Z, mtr.getTarget(0))
#     scores.append(model.score(Z, mtr.getTarget(0)))
  
# plt.title("Effect of n_iter")
# plt.xlabel("n_iter")
# plt.ylabel("score")
# plt.plot(n_iters, scores) 
# plt.show()

Loaded MyTotoResearch algo_no:  1
1521
(1521, 7)
0.0
 Time taken:  4.7999999999603915e-05  


In [28]:

def getAllData(df):
    drop_cols = ['T', 'L','M','S','R','E','A','V' ,'J','U','K']
    X = df.drop(drop_cols, axis=1)
    return X

test_data = mtr.get_test_data()
X = mtr.modified_dataset(getAllData(test_data)) #

scaler = RobustScaler()
scaler.fit(X)
Z = scaler.transform(X)

#Z = X
#X = mtr.modified_dataset(getAdjustedDataF(test_data,f))
predictions = multi_target_sgd.predict(Z)
dfResult= pd.DataFrame(predictions, columns=['N1', 'N2', 'N3', 'N4', 'N5','N6', 'N7'])
mtr.print_predictions(dfResult)




20180514   [17 24 29 45 46 49  5]   [10. 36. 34. 10. 45. 27.]   [45]
20180517   [ 7 21 25 29 35 37 13]   [10.  6. 34. 38. 29. 27.]   [29]
20180521   [ 8 10 16 30 37 44 17]   [16.  6. 34. 10. 29. 27.]   [10 16]
20180524   [11 25 26 34 36 42 16]   [10.  6. 34. 38. 45. 27.]   [34]
20180528   [ 5  9 27 28 30 44  2]   [10.  6. 34. 10. 29. 27.]   [27]
20180531   [11 13 24 26 47 49 33]   [14.  6. 34. 17. 29. 44.]   []
20180604   [20 22 31 37 43 45 27]   [14. 35. 14. 38. 29. 44.]   []
20180607   [12 20 29 31 37 39 42]   [14. 35. 34. 36. 32. 27.]   []
20180611   [16 25 30 37 44 49 34]   [14. 36. 28. 36. 29. 44.]   [44]
20180614   [ 4 29 31 35 42 48  1]   [16. 35. 14. 38. 29. 44.]   [29 35]
20180618   [11 15 22 23 26 43 25]   [10.  6. 14. 17. 22. 27.]   [22]
20180621   [ 4  6 15 24 30 35 46]   [ 6. 11. 21.  9. 31. 46.]   [ 6 46]
20180625   [ 2  5 25 38 44 48  9]   [15. 18. 21. 27. 40. 40.]   []
20180628   [ 2  7 22 27 40 47 48]   [ 7. 11. 14. 32. 41. 46.]   [7]
20180702   [12 13 26 33 35 38 23] 

In [ ]:
2 9 14 36 46 48
5